In [1]:
import urllib
import PIL.Image
import tensorflow_docs.plots

import numpy as np
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorflow_docs as tfdocs
import tensorflow_datasets as tfds

from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
mpl.rcParams['figure.figsize'] = (12, 5)

In [ ]:
# Example Image

image_path = tf.keras.utils.get_file("cat.jpg", "https://storage.googleapis.com/download.tensorflow.org/example_images/320px-Felis_catus-cat_on_snow.jpg")
PIL.Image.open(image_path)

In [ ]:
image_string = tf.io.read_file(image_path)
image = tf.image.decode_jpeg(image_string, channels = 3)

In [ ]:
def visualize(original, augmented):
  fig = plt.figure()
  plt.subplot(1,2,1)
  plt.title('Original image')
  plt.imshow(original)

  plt.subplot(1,2,2)
  plt.title('Augmented image')
  plt.imshow(augmented)

In [ ]:
# Flip Image

flipped = tf.image.flip_left_right(image)
visualize(image, flipped)

In [ ]:
# Grayscale Image

grayscaled = tf.image.rgb_to_grayscale(image)
visualize(image, tf.squeeze(grayscaled))
plt.colorbar()

In [ ]:
# Saturate the Image

saturated = tf.image.adjust_saturation(image, 3)
visualize(image, saturated)

In [ ]:
# Change Image Brightness

bright = tf.image.adjust_brightness(image, 0.4)
visualize(image, bright)

In [ ]:
# Rotate Image

rotated = tf.image.rot90(image)
visualize(image, rotated)

In [ ]:
# Center Crop the image

cropped = tf.image.central_crop(image, central_fraction = 0.5)
visualize(image,cropped)